In [27]:
#Import Required libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup


In [5]:
finviz_url = "https://finviz.com/quote.ashx?t="
ticker = "TSLA"
url = finviz_url + ticker

In [25]:
req = Request(url=url, headers={'user-agent': 'my-app'})
response = urlopen(req)

news_table = {}
html = BeautifulSoup(response,html)
news_table = html.find(id='news-table')
# news_table

dataRows = news_table.findAll('tr')


In [12]:
dataRows

[<tr><td align="right" style="white-space:nowrap" width="130">Jul-21-22 02:44AM  </td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/news/bitcoin-dragged-down-further-weak-064459040.html" target="_blank">Bitcoin Dragged Down Further on Weak Stocks in Asia, Tesla Sales</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> Bloomberg</span></div></div></td></tr>,
 <tr><td align="right" width="130">02:30AM  </td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://www.fool.com/earnings/call-transcripts/2022/07/21/tesla-tsla-q2-2022-earnings-call-transcript/?source=eptyholnk0000202&amp;utm_source=yahoo-host&amp;utm_medium=feed&amp;utm_campaign=article" target="_blank">Tesla (TSLA) Q2 2022 Earnings Call Transcript</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> Motley Fool</span></di

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(columns=['News_Title', 'Time'])

In [15]:
for i, table_row in enumerate(dataRows):
    a_text = table_row.a.text
    td_text = table_row.td.text
    
    df = df.append({'News_Title': a_text, 'Time': td_text}, ignore_index=True)

In [16]:
df

,News_Title,Time
0,Bitcoin Dragged Down Further on Weak Stocks in...,Jul-21-22 02:44AM
1,Tesla (TSLA) Q2 2022 Earnings Call Transcript,02:30AM
2,"Stocks, Futures Rise on Gas Deliveries; Euro G...",02:21AM
3,"Tesla sells 75% of its Bitcoin holdings, but n...",01:46AM
4,"Lithium Refining Is a License to Print Money, ...",12:55AM
...,...,...
95,Dow Jones Rallies But IBM Dives On Earnings; N...,09:55AM
96,Earnings season to offer a huge unveiling of c...,09:37AM
97,"Twitter, Tesla CEO Elon Musk go to court for f...",09:31AM
98,10 Most Shorted Stocks to Watch in July,09:26AM


In [17]:
#Import Dependencies for sentiment analysis using our neural network
import tensorflow as tf
import numpy
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [18]:
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
model = tf.keras.models.load_model('./model1.h5')

In [19]:
oov_tok = '<OOV>'
trunc_type = 'post'
padding_type='post'
vocab_size =1000
max_length = 142

def preprocessText(text):
    sequences = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type,
                          truncating=trunc_type)
    return padded

prep = preprocessText(df.News_Title)

In [20]:
prep = model.predict(prep)

In [21]:
prep

array([[0.24980627, 0.15260766, 0.59758604],
       [0.10735555, 0.5427653 , 0.34987915],
       [0.04537959, 0.77532816, 0.17929226],
       [0.01860556, 0.90871096, 0.07268352],
       [0.07424019, 0.62789077, 0.2978691 ],
       [0.05614398, 0.7459152 , 0.19794083],
       [0.12132297, 0.45182538, 0.42685172],
       [0.09443878, 0.5497185 , 0.3558427 ],
       [0.071904  , 0.64417946, 0.28391656],
       [0.03359186, 0.85352254, 0.11288567],
       [0.09359716, 0.6044126 , 0.3019902 ],
       [0.16428156, 0.33045286, 0.50526553],
       [0.02627007, 0.8641242 , 0.10960582],
       [0.0559708 , 0.7488203 , 0.19520885],
       [0.24180005, 0.08864424, 0.6695557 ],
       [0.06864962, 0.715281  , 0.2160694 ],
       [0.12852213, 0.4157472 , 0.45573068],
       [0.0979137 , 0.5723016 , 0.32978466],
       [0.10274094, 0.46290648, 0.43435252],
       [0.05648609, 0.74509585, 0.19841802],
       [0.0974074 , 0.560474  , 0.34211865],
       [0.04185587, 0.78802663, 0.17011748],
       [0.

In [22]:
df['sent'] = np.argmax(prep, axis=-1) #to get our sentiment

In [23]:
df

,News_Title,Time,sent
0,Bitcoin Dragged Down Further on Weak Stocks in...,Jul-21-22 02:44AM,2
1,Tesla (TSLA) Q2 2022 Earnings Call Transcript,02:30AM,1
2,"Stocks, Futures Rise on Gas Deliveries; Euro G...",02:21AM,1
3,"Tesla sells 75% of its Bitcoin holdings, but n...",01:46AM,1
4,"Lithium Refining Is a License to Print Money, ...",12:55AM,1
...,...,...,...
95,Dow Jones Rallies But IBM Dives On Earnings; N...,09:55AM,2
96,Earnings season to offer a huge unveiling of c...,09:37AM,2
97,"Twitter, Tesla CEO Elon Musk go to court for f...",09:31AM,1
98,10 Most Shorted Stocks to Watch in July,09:26AM,1


In [24]:
df.sent.value_counts()

1    71
2    29
Name: sent, dtype: int64

In [28]:
pip install alpaca_trade_api


  Using cached alpaca_trade_api-2.3.0-py3-none-any.whl (33 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.2/574.2 kB 5.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.2/97.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 3.6 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.2
    Uninstalling msgpack-1.0.2:
      Successfully uninstalled msgpack-1.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Unins